In [1]:
import sys
sys.path.insert(0, '..')

In [2]:
domain = 'HPC-FAIR'

In [3]:
# Build grammar graph
import HISyn.domain_knowledge.DomainKnowledgeConstructor as dkc
gg = dkc.set_grammar_graph(domain, './Documentation/' + domain + '/grammar-gen.txt', './Documentation/' + domain + '/API_documents.txt', reload = True)

-- Log: Set grammar graph...     

-- Log: Reading grammar file and pre-processing text...     

-- Log: Grammar text preparation finished     

-- Log: Start building grammar tree...     

-- Log: Add API description...     

-- Log: Set nodes max formal children length     

-- Log: Grammar tree build finished.     

-- Log: Grammar graph built     



In [4]:
gg.build_test()

BuildingDataset ['_output_log'] {'_kernel_data_best_csv', '_kernel_stat_csv'} Merge kernel-level and data-level features by outer-join norm         
ModelTraining ['_GPUTrace_csv', '_dataset_csv'] {'_mergedDataSet_csv'} collect & sort time measurements from all variants and report the top varaiant for each each kernel and data input norm         
ModelEvaluation ['_kernel_data_best_csv', '_mergedDataSet_csv'] {'_labelledData_csv'} Adding the label of best-performing variants to the merged data norm         
DataPreProcessing ['_GPUTraceOutput_log'] {'_GPUTrace_csv'} collect & filter baseline data-level features norm         
DataCollection ['_nsight_log'] {'_dataset_csv'} collect baseline kernel-level features norm         
_root [['_kernel_stat_csv'], ['_labelledData_csv']] set() ['_kernel_stat_csv', '_labelledData_csv']          
_kernel_data_best_csv [['BuildingDataset']] {'ModelEvaluation'} ['BuildingDataset(_output_log)']          
_kernel_stat_csv [['BuildingDataset']] {'_root'} 

In [5]:
# set NL query
text = 'find the labelled data'

# get query from test cases
import HISyn.front_end.front_end_function_kit as front_kit
# index = 0
# text = front_kit.read_text('./Documentation/' + domain + '/text_new.txt', index)

In [6]:
# parsing the query and prune the unimportant edges.
# NLP
get_ipython().run_line_magic('env', 'CORENLP_HOME=../HISyn/third_party_pkgs/stanford-corenlp-full-2018-10-05')
nlp = front_kit.nlp_parsing(text, domain)

env: CORENLP_HOME=../HISyn/third_party_pkgs/stanford-corenlp-full-2018-10-05
-- Log: Parsing text...     

-- Log: Check NLP server status...     

-- Log: NLP server already started!     

test:  find the labelled data            
-- Log: Start parsing sentence: find the labelled data     

-- Log: starting JAVA Stanford CoreNLP Server...     

-- Log: Parsing finished     



In [7]:
front_kit.domain_specfic_parsing_rules(domain, gg, nlp)

-- Log: apply domain_specific_parsing_rules...     

-- Log: no domain specific rules.     



In [8]:
import HISyn.common_knowledge.NLPCommonKnowledge as nlpck
front_kit.prune_edges(nlp, nlpck.prunable_dep_tags, nlpck.prunable_pos_tags, nlpck.common_knowledge_tags)

-- Log: Pruning unimportant edges...     

-- Log: Pruning unimportant edges...     



In [9]:
nlp.displayByEdge()

---------------- Dependency graph -----------------             
find [VB] [O] [find] [] --dobj--> data [NNS] [O] [datum] [] ====>> [] []
             
data [NNS] [O] [datum] [] --amod--> labelled [VBN] [O] [label] [] ====>> [] []
             
--------------------------------------------------             


In [10]:
import HISyn.back_end.back_end_function_kit as back_kit

In [11]:
back_kit.semantic_mapping(domain, gg, nlp, nlpck.common_knowledge_tags)

-- Log: checking common knowledge     

-- Log: common knowledge replace finished     

-- Log: start keywords mapping     

find ['merge', 'kernel-level', 'and', 'data-level', 'features', 'by', 'outer-join'] False
find ['collect', '&', 'sort', 'time', 'measurements', 'from', 'all', 'variants', 'and', 'report', 'the', 'top', 'varaiant', 'for', 'each', 'each', 'kernel', 'and', 'data', 'input'] False
find ['adding', 'the', 'label', 'of', 'best-performing', 'variants', 'to', 'the', 'merged', 'data'] False
find ['collect', '&', 'filter', 'baseline', 'data-level', 'features'] False
find ['collect', 'baseline', 'kernel-level', 'features'] False
find ['merge', 'kernel-level', 'and', 'data-level', 'features', 'by', 'outer-join'] False
find ['collect', '&', 'sort', 'time', 'measurements', 'from', 'all', 'variants', 'and', 'report', 'the', 'top', 'varaiant', 'for', 'each', 'each', 'kernel', 'and', 'data', 'input'] False
find ['adding', 'the', 'label', 'of', 'best-performing', 'variants', 'to', '

In [12]:
nlp.displayByEdge()

---------------- Dependency graph -----------------             
find [VB] [O] [find] [] --dobj--> data [NNS] [O] [datum] ['ModelTraining', 'ModelEvaluation'] ====>> [] []
             
data [NNS] [O] [datum] ['ModelTraining', 'ModelEvaluation'] --amod--> labelled [VBN] [O] [label] ['ModelEvaluation'] ====>> [] []
             
--------------------------------------------------             


In [13]:
back_kit.longest_matching(nlp)

-- Log: start longest matching     

-- Log: remove empty edges     

-- Log: empty edge removed.     



In [14]:
nlp.displayByEdge()

---------------- Dependency graph -----------------             
find [VB] [O] [find] [] --dobj--> data [NNS] [O] [datum] ['ModelEvaluation'] ====>> [] []
             
--------------------------------------------------             


In [15]:
[dependent_dict, no_gov_source_dict, empty_mapping_source_edge_dict, root_index] = back_kit.reordering(nlp, gg, nlpck.preposition)

-- Log: set preposition mappings ...     

-- Log: subj reordering ...     

-- Log: checking no governor dependency source     

-- Log: add root token to no governor source     



In [16]:
nlp.displayByEdge()

---------------- Dependency graph -----------------             
root [] [] [] [] --dobj--> data [NNS] [O] [datum] ['ModelEvaluation'] ====>> [] []
             
--------------------------------------------------             


In [17]:
back_kit.reversed_all_path_searching(domain, nlp, gg, dependent_dict)

-- Log: Starting all paths search ...     

-- Log: singel start BFS...     

-- Log: Start API:  ModelEvaluation    

-- Log: endAPIset:  []    

-- Log: path_limit:  None    

-- Log: total paths:  1    

-- Log: Edges reordering...      

-- Log: replace common knowledge api...     



In [18]:
nlp.displayByEdge()

---------------- Dependency graph -----------------             
root [] [] [] [] --dobj--> data [NNS] [O] [datum] ['ModelEvaluation'] ====>> [['ModelEvaluation']] []
             
--------------------------------------------------             


In [19]:
final_cgt_list = back_kit.path_selection_and_combination(nlp, gg, dependent_dict, root_index)

-- Log: find siblings on dependency graph...     

-- Log: attach_non_sibling_edges...     

-- Log: Path selection: min path/prefix tree heuristic     

-- Log: connect all cgts whose root is not root     

-- Log: connect cgts to root cgts     

-- Log: Set all cgt combinations ...     

-- Log: Combine cgts in each group ...     



In [20]:
for t in final_cgt_list:
    t.display()

| root |  |         
--| _root |  |         
----| _labelledData_csv |  |         
------| ModelEvaluation |  |         


In [21]:
[min_expr, expr_list] = back_kit.code_generation(gg, final_cgt_list)

-- Log: checking grammar on cgts...     

-- Log: changing argument order of valid nodes     

-- Log: removing trees with wrong grammar...     

-- Log: adding default argument to complete the cgt...     

-- Log: checking grammar on cgts...     

-- Log: changing argument order of valid nodes     

-- Log: converting to codes...     

-- Log: converting to codes...     

-- Log: promising exprs:  ['ModelEvaluation(BuildingDataset(_output_log),ModelTraining(DataPreProcessing(_GPUTraceOutput_log),DataCollection(_nsight_log)))']    

-- Log: full candidate list:  1    

